In [1]:
! pip install transformers
! pip install datasets
! pip install scipy sklearn
! pip install huggingface_hub
! pip install metric
! pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 23.7 MB/s eta 0:00:00
  Preparing metadata (set

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS and setup Git if you haven't already. Uncomment the following instructions and adapt with your name and email:

In [3]:
!apt install git-lfs
!git config --global user.email "shubhampilivkar@gmail.com"
!git config --global user.name "Shubham"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [4]:
import transformers
print(transformers.__version__)

4.38.2


In [5]:
from transformers.utils import send_example_telemetry
send_example_telemetry("text_classification_notebook", framework="tensorflow")

# Fine-tuning a model on a text classification task

In [6]:
from transformers import BertTokenizerFast

In [7]:
# model_checkpoint = "distilbert-base-uncased"
model_checkpoint="bert-base-uncased"
batch_size = 16

In [8]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv("/content/news_category_classification.csv")

In [9]:
remove_words = ["â€™"]

def remove_words_from_text(text):
    for word in remove_words:
        text = text.replace(word, "")
    return text

df['headlines'] = df['headlines'].apply(remove_words_from_text)

In [10]:
import re
import string

def clean_text(text):
    text = str(text).lower()
    text = re.sub('https?://\S+|www\.\S+', ' ', text) # Remove URL from string
    text = re.sub('<.*?>+', ' ', text) # Remove HTML Tags
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text) # Replace Punctuation with space
    text = re.sub('\n', ' ', text) # Replace NewLine with space
    text = re.sub(r'\w*\d\w*', ' ', text) # Remove alphanumeric
    text = text.replace('-', ' ')
    text = text.replace('’’', '').replace('’', '').replace('‘‘', '').replace('‘', '')
    text = ' '.join(text.split())  # Strip extra spaces
    return text

In [11]:
df['headlines'] = df['headlines'].apply(clean_text)

In [12]:
remove_words = ["Ã¯Â¿Â½", "check", "CHECK", "Â", "Ã¢Â€Â˜", "Ã¢Â€Â™", "Â’ Â", "Â‘", "Â‘"]

def remove_words_from_text(text):
    for word in remove_words:
        text = text.replace(word, "")
    return text

df['headlines'] = df['headlines'].apply(remove_words_from_text)

In [13]:
df.to_csv("News_Classification_data.csv", index=False)

In [14]:
dataset = Dataset.from_pandas(df)

In [15]:
dataset

Dataset({
    features: ['headlines', 'category'],
    num_rows: 10000
})

In [16]:
dataset[0]

{'headlines': 'nirmala sitharaman to equal morarji desais record with her sixth straight budget',
 'category': 'business'}

In [17]:
from transformers import AutoTokenizer
# tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [18]:
tokenizer("Hello, this is a sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2003, 1037, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [19]:
classes = list(set(dataset['category']))
classes_dict = {classes[i]: i for i in range(len(classes))}
reverse_dict_classes = {i:classes[i] for i in range(len(classes))}
num_classes = len(classes_dict)
print("Number of categorys:", num_classes)
print("categorys:", classes_dict)

Number of categorys: 5
categorys: {'education': 0, 'entertainment': 1, 'technology': 2, 'business': 3, 'sports': 4}


In [20]:
def preprocess(dataset):
  return {'headlines': dataset['headlines'], 'label': classes_dict[dataset['category']]}

In [21]:
dataset = dataset.map(preprocess)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [22]:
def preprocess_function(examples):
    return tokenizer(examples['headlines'], padding=True) #, truncation=True, max_length=30

In [23]:
pre_tokenizer_columns = set(dataset.features)
encoded_dataset = dataset.map(preprocess_function, batched=True)
tokenizer_columns = list(set(encoded_dataset.features) - pre_tokenizer_columns)
print("Columns added by tokenizer:", tokenizer_columns)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Columns added by tokenizer: ['token_type_ids', 'input_ids', 'attention_mask']


In [24]:
encoded_dataset.features["label"]

Value(dtype='int64', id=None)

In [25]:
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf
num_labels = 5
{'education': 0, 'entertainment': 1, 'technology': 2, 'business': 3, 'sports': 4}
label2id = {'education': 0, 'entertainment': 1, 'technology': 2, 'business': 3, 'sports': 4}
id2label = {0: "education", 1: "entertainment", 2: "technology", 3: "business", 4: "sports"}

model = TFAutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
tf_dataset = model.prepare_tf_dataset(
    encoded_dataset,
    shuffle=True,
    batch_size=16,
    tokenizer=tokenizer
)

In [27]:
train_dataset = tf_dataset.take(int(0.8*len(tf_dataset)))
val_dataset = tf_dataset.skip(int(0.8*len(tf_dataset)))

In [28]:
from transformers import create_optimizer

num_epochs = 5
batches_per_epoch = len(train_dataset) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)

optimizer, schedule = create_optimizer(
    init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps
)
model.compile(optimizer=optimizer)

In [29]:
from datasets import load_dataset
from evaluate import load

In [30]:
import numpy as np
metric = load("glue", "mrpc")

In [31]:
fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

{'accuracy': 0.453125, 'f1': 0.38596491228070173}

In [32]:
import evaluate

In [33]:
from transformers.keras_callbacks import KerasMetricCallback
from datasets import load_metric

metric = load_metric('accuracy', average='weighted')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

metric_callback = KerasMetricCallback(
    metric_fn=compute_metrics, eval_dataset=val_dataset
)

<ipython-input-33-cf9f803621fd>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('accuracy', average='weighted')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [34]:
from transformers.keras_callbacks import PushToHubCallback
from tensorflow.keras.callbacks import TensorBoard

model_name = model_checkpoint.split("/")[-1]
push_to_hub_model_id = f"{model_name}-finetuned-news_category_classification"

tensorboard_callback = TensorBoard(log_dir="./news_category_classification_model_save/logs")

push_to_hub_callback = PushToHubCallback(
    output_dir="./news_category_classification_model_save",
    tokenizer=tokenizer,
    hub_model_id=push_to_hub_model_id,
)

callbacks = [metric_callback, tensorboard_callback, push_to_hub_callback]

model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=num_epochs,
    callbacks=callbacks,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/DriveMyScream/bert-base-uncased-finetuned-news_category_classification into local empty directory.


Epoch 1/5


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
500/500 [==============================] - 245s 304ms/step - loss: 0.4760 - val_loss: 0.2983 - accuracy: 0.9385
Epoch 2/5
500/500 [==============================] - 122s 244ms/step - loss: 0.3600 - val_loss: 0.3235 - accuracy: 0.9350
Epoch 3/5
500/500 [==============================] - 121s 241ms/step - loss: 0.3579 - val_loss: 0.3197 - accuracy: 0.9450
Epoch 4/5
500/500 [==============================] - 124s 248ms/step - loss: 0.3569 - val_loss: 0.3056 - accuracy: 0.9480
Epoch 5/5
500/500 [==============================] - 125s 251ms/step - loss: 0.3572 - val_loss: 0.3085 - accuracy: 0.9495


In [35]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("DriveMyScream/bert-base-uncased-finetuned-news_category_classification")
tokenizer = AutoTokenizer.from_pretrained("DriveMyScream/bert-base-uncased-finetuned-news_category_classification")

config.json:   0%|          | 0.00/902 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some layers from the model checkpoint at DriveMyScream/bert-base-uncased-finetuned-news_category_classification were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at DriveMyScream/bert-base-uncased-finetuned-news_category_classification.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predict

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [36]:
sentences = [
    "The judge told the jurors to think carefully.",
    "The judge told that the jurors to think carefully."
]

In [37]:
tokenized = tokenizer(sentences, return_tensors="np", padding="longest")

outputs = model(tokenized).logits

classifications = np.argmax(outputs, axis=1)
print(classifications)

[2 2]


In [38]:
classifications = [model.config.id2label[output] for output in classifications]
print(classifications)

['technology', 'technology']


## Pipeline API

In [39]:
from transformers import pipeline
classifier = pipeline("text-classification", "DriveMyScream/bert-base-uncased-finetuned-news_category_classification", framework="tf")

Some layers from the model checkpoint at DriveMyScream/bert-base-uncased-finetuned-news_category_classification were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at DriveMyScream/bert-base-uncased-finetuned-news_category_classification.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predict

In [40]:
classifier(sentences)

[{'label': 'technology', 'score': 0.2765316069126129},
 {'label': 'technology', 'score': 0.2505176365375519}]